# 🏎️ Formula 1 Tire Change Prediction - Quick Start (Colab Pro)

**Sistema completo di predizione cambi gomme F1 con RNN multi-task**

Questo notebook ti guida attraverso il setup completo e il training del modello ottimizzato per Google Colab Pro.

## 📋 Pre-requisiti
- Google Colab Pro (raccomandato)
- Dati F1 caricati su Google Drive
- GPU abilitata (Runtime > Change runtime type > GPU)

---

## 🚀 Step 1: Setup Ambiente

Iniziamo con il setup automatico dell'ambiente ottimizzato per Colab Pro.

In [ ]:
# Clona repository (se necessario) o carica files
# NOTA: Se stai usando questo notebook, presumiamo che i file siano già caricati

import os
import sys

# Verifica che siamo nella directory corretta
if not os.path.exists('colab_models'):
    print("❌ Directory colab_models non trovata!")
    print("🔧 Assicurati di aver caricato tutti i file nella directory corretta")
    print("📁 Struttura attesa:")
    print("   /content/colab_models/")
    print("   ├── setup_colab_pro.py")
    print("   ├── configs/")
    print("   ├── models/")
    print("   └── ...")
else:
    print("✅ Directory colab_models trovata!")
    
# Cambia directory
%cd /content/colab_models

In [ ]:
# Esegui setup automatico
print("🚀 Avvio setup automatico per Colab Pro...")
%run setup_colab_pro.py

## 📊 Step 2: Unificazione Dati

I dati F1 sono attualmente distribuiti in multiple cartelle. Li unifichiamo in un dataset completo.

In [ ]:
# Verifica dati disponibili su Drive
from google.colab import drive
import os

# Mount Drive se non già fatto
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Check directories
domenico_path = "/content/drive/MyDrive/domenicoDL"
vincenzo_path = "/content/drive/MyDrive/Vincenzo/processed_races"

print("📁 Controllo dati disponibili:")
print(f"DomenicoDL: {os.path.exists(domenico_path)} - {len(os.listdir(domenico_path)) if os.path.exists(domenico_path) else 0} files")
print(f"Vincenzo: {os.path.exists(vincenzo_path)} - {len(os.listdir(vincenzo_path)) if os.path.exists(vincenzo_path) else 0} files")

if not os.path.exists(domenico_path) and not os.path.exists(vincenzo_path):
    print("⚠️  Nessuna cartella dati trovata! Assicurati di aver caricato i dati su Drive.")

In [ ]:
# Unifica tutti i dati in un dataset completo
from data.data_unifier_complete import CompleteDataUnifier

print("🔄 Avvio unificazione dati...")
print("⏱️  Questo processo può richiedere 10-15 minuti")

unifier = CompleteDataUnifier()
dataset = unifier.unify_all_data()

print(f"✅ Unificazione completata!")
print(f"📊 Dataset finale: {len(dataset):,} righe, {len(dataset.columns)} colonne")

In [ ]:
# Esplorazione rapida del dataset unificato
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("🔍 Esplorazione dataset unificato:")
print(f"Shape: {dataset.shape}")
print(f"Anni: {sorted(dataset['Year'].unique()) if 'Year' in dataset.columns else 'N/A'}")
print(f"Piloti: {dataset['Driver'].nunique() if 'Driver' in dataset.columns else 'N/A'}")
print(f"GP: {dataset['EventName'].nunique() if 'EventName' in dataset.columns else 'N/A'}")

# Visualizzazione distribuzione anni
if 'Year' in dataset.columns:
    plt.figure(figsize=(10, 6))
    dataset['Year'].value_counts().sort_index().plot(kind='bar')
    plt.title('Distribuzione Dati per Anno')
    plt.xlabel('Anno')
    plt.ylabel('Numero di Record')
    plt.xticks(rotation=45)
    plt.show()

## 🏋️ Step 3: Training del Modello

Avviamo il training completo del modello LSTM multi-task ottimizzato per Colab Pro.

In [ ]:
# Verifica GPU e memoria
import torch
import psutil

print("🔧 Controllo risorse sistema:")

# GPU
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU: {gpu_name} ({gpu_memory:.1f}GB)")
    
    if "T4" in gpu_name or "A100" in gpu_name or "V100" in gpu_name:
        print("🚀 GPU ottima per training!")
    else:
        print("⚠️  GPU rilevata ma potrebbe essere lenta")
else:
    print("❌ GPU non disponibile! Abilita GPU in Runtime > Change runtime type")

# RAM
memory = psutil.virtual_memory()
total_gb = memory.total / 1e9
available_gb = memory.available / 1e9

print(f"✅ RAM: {total_gb:.1f}GB totale, {available_gb:.1f}GB disponibile")

if total_gb > 20:
    print("🚀 RAM eccellente per Colab Pro!")
elif total_gb > 12:
    print("✅ RAM buona per training")
else:
    print("⚠️  RAM limitata, considera Colab Pro")

In [ ]:
# Setup trainer
from training.train_from_scratch_pro import ProTrainer

print("🏋️ Inizializzazione trainer...")

# Crea trainer con configurazione ottimizzata
trainer = ProTrainer(
    config_path="configs/model_config_pro.yaml",
    project_dir="/content/drive/MyDrive/F1_TireChange_Project"
)

print("✅ Trainer inizializzato!")
print(f"📱 Device: {trainer.device}")
print(f"📁 Model dir: {trainer.model_dir}")
print(f"📊 Results dir: {trainer.results_dir}")

In [ ]:
# AVVIO TRAINING COMPLETO
print("🚀 AVVIO TRAINING DA ZERO...")
print("⏱️  Tempo stimato: 4-6 ore su Colab Pro")
print("💾 Checkpoint automatici ogni 30 minuti")
print("🔄 Puoi interrompere e riprendere il training")
print("="*60)

# Training con parametri ottimizzati
results = trainer.train_complete(max_epochs=100)

print("="*60)
print("🎉 TRAINING COMPLETATO!")
print(f"⏱️  Tempo totale: {results['total_training_time']/3600:.1f} ore")
print(f"🎯 Best validation loss: {results['best_val_loss']:.4f}")

## 📊 Step 4: Analisi Risultati

Analizziamo i risultati del training e le performance del modello.

In [ ]:
# Visualizza curve di training
import matplotlib.pyplot as plt
import numpy as np

history = results['training_history']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss curves
axes[0,0].plot(history['epochs'], history['train_loss'], label='Train Loss', color='blue')
axes[0,0].plot(history['epochs'], history['val_loss'], label='Val Loss', color='red')
axes[0,0].set_title('Training & Validation Loss')
axes[0,0].set_xlabel('Epoch')
axes[0,0].set_ylabel('Loss')
axes[0,0].legend()
axes[0,0].grid(True)

# Accuracy curves
axes[0,1].plot(history['epochs'], history['train_acc'], label='Train Acc', color='blue')
axes[0,1].plot(history['epochs'], history['val_acc'], label='Val Acc', color='red')
axes[0,1].set_title('Training & Validation Accuracy')
axes[0,1].set_xlabel('Epoch')
axes[0,1].set_ylabel('Accuracy')
axes[0,1].legend()
axes[0,1].grid(True)

# Learning rate
axes[1,0].plot(history['epochs'], history['learning_rates'], color='green')
axes[1,0].set_title('Learning Rate Schedule')
axes[1,0].set_xlabel('Epoch')
axes[1,0].set_ylabel('Learning Rate')
axes[1,0].set_yscale('log')
axes[1,0].grid(True)

# Final metrics
test_metrics = results['final_test_metrics']
metrics_names = list(test_metrics.keys())
metrics_values = list(test_metrics.values())

axes[1,1].bar(metrics_names, metrics_values, color='skyblue')
axes[1,1].set_title('Final Test Metrics')
axes[1,1].set_ylabel('Score')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("📊 Risultati Finali:")
for metric, value in test_metrics.items():
    print(f"  {metric}: {value:.4f}")

In [ ]:
# Informazioni modello finale
model_info = trainer.model.get_model_info()

print("🧠 Informazioni Modello:")
print(f"  Parametri totali: {model_info['total_parameters']:,}")
print(f"  Parametri LSTM: {model_info['lstm_parameters']:,}")
print(f"  Parametri Shared Trunk: {model_info['shared_trunk_parameters']:,}")
print(f"  Input shape: {model_info['input_shape']}")
print(f"  Hidden size: {model_info['hidden_size']}")
print(f"  Num layers: {model_info['num_layers']}")

# Calcola dimensione modello in MB
model_size_mb = model_info['total_parameters'] * 4 / 1024 / 1024  # 4 bytes per parameter
print(f"  Dimensione modello: {model_size_mb:.1f} MB")

## 🚀 Step 5: Test Inference (Opzionale)

Testiamo il modello addestrato con alcune predizioni di esempio.

In [ ]:
# Test rapido inference
import torch
import numpy as np

# Carica best model
best_model_path = "/content/drive/MyDrive/F1_TireChange_Project/models/checkpoints/best_model.pth"

if os.path.exists(best_model_path):
    print("🔄 Caricamento best model...")
    
    checkpoint = torch.load(best_model_path)
    trainer.model.load_state_dict(checkpoint['model_state_dict'])
    trainer.model.eval()
    
    print("✅ Modello caricato!")
    
    # Test con dati dummy
    print("🧪 Test inference con dati di esempio...")
    
    # Simula sequenza di 10 timesteps con 52 features
    dummy_input = torch.randn(1, 10, 52).to(trainer.device)
    
    with torch.no_grad():
        outputs = trainer.model(dummy_input)
        
        # Predizioni
        tire_change_prob = torch.sigmoid(outputs['tire_change_logits']).cpu().item()
        tire_type_probs = torch.softmax(outputs['tire_type_logits'], dim=-1).cpu().numpy()[0]
    
    print(f"🎯 Predizione cambio gomme: {tire_change_prob:.3f} ({tire_change_prob*100:.1f}%)")
    print(f"🏎️ Probabilità tipi gomme:")
    
    # Compound types (assumendo ordine standard)
    compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTER', 'WET', 'HYPER', 'ULTRA', 'SUPER', 'SUPERHARD']
    for i, (compound, prob) in enumerate(zip(compounds, tire_type_probs)):
        print(f"    {compound}: {prob:.3f} ({prob*100:.1f}%)")
    
    print("\n✅ Test inference completato!")
    print("🎉 Il modello è pronto per essere usato!")
    
else:
    print("❌ Best model non trovato. Assicurati che il training sia completato.")

## 🎯 Next Steps

Congratulazioni! Hai completato con successo il training del modello di predizione cambi gomme F1.

### 📁 Files Salvati
- **Best Model**: `/content/drive/MyDrive/F1_TireChange_Project/models/checkpoints/best_model.pth`
- **Training Logs**: `/content/drive/MyDrive/F1_TireChange_Project/results/training_logs/`
- **Dataset Unificato**: `/content/drive/MyDrive/F1_TireChange_Project/data/unified/f1_complete_dataset.parquet`

### 🚀 Prossimi Step
1. **Valutazione Avanzata**: Esegui `02_data_exploration.ipynb` per analisi approfondite
2. **Ottimizzazione**: Usa `05_hyperparameter_opt.ipynb` per tuning parametri
3. **Demo Real-time**: Prova `06_real_time_demo.ipynb` per inference live
4. **Production**: Setup API con gli script nella cartella `inference/`

### 📞 Support
- Consulta il `README.md` per guide dettagliate
- Check troubleshooting section per problemi comuni
- Monitora memoria e GPU durante l'uso

**Happy Racing! 🏁**